Create procedure that trains models

In [ ]:
DROP PROCEDURE IF EXISTS generate_model;
GO

CREATE PROCEDURE generate_model (@trained_model varbinary(max) OUTPUT)
AS 
BEGIN    
    EXECUTE sp_execute_external_script
            @language = N'Python',
            @script = N'
import pandas
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle

raw_dataframe = input_data

# Data pre-processing
raw_dataframe.town = pandas.factorize(raw_dataframe.town)[0]
raw_dataframe.flat_type = pandas.factorize(raw_dataframe.flat_type)[0]
raw_dataframe.flat_model = pandas.factorize(raw_dataframe.flat_model)[0]

# Data range refinement
town_value = 0
month_value = 1
year_value = 2017
refined_df = raw_dataframe.loc[ (raw_dataframe[''town''] == town_value) &
                              # (raw_dataframe[''month''] == month_value) &
                                (raw_dataframe[''year''] == year_value) ]
dependent_variable = [''resale_price''] 
dependent_dataset = refined_df[dependent_variable]

# Define dependent and independent variables
independent_variables = [''flat_type'',''floor_area_sqm'',''flat_model'',''remaining_lease_months''] #input any number of variables
dependent_variable = [''resale_price''] #only input one variable

# Split data into dependent and independent 
independent_dataset = refined_df[independent_variables]
dependent_dataset = refined_df[dependent_variable]

# Further splitting dataset into training and testing subsets
test_ratio=0.2 #splits the data into testing and training sets with ratio 0.2
indp_train_set, indp_test_set, dep_train_set, dep_test_set = train_test_split(independent_dataset, 
                                                                              dependent_dataset, 
                                                                              test_size=test_ratio)


# Create model object & train model
linear_model = LinearRegression()
linear_model.fit(indp_train_set, dep_train_set)

# Output trained model
trained_model = pickle.dumps(linear_model)
'
, @input_data_1 = N'SELECT * FROM [bdc].[hdb_resale_prices]'
, @input_data_1_name = N'input_data'
, @params = N'@trained_model varbinary(max) OUTPUT'
, @trained_model = @trained_model OUTPUT;
END; 
GO

Create a table to store trained models

In [ ]:
Use externalDB;
DROP TABLE IF EXISTS [bdc].[hdb_py_models];
GO
CREATE TABLE [bdc].[hdb_py_models](
    model_name VARCHAR(30) NOT NULL DEFAULT('default model') PRIMARY KEY,
    model VARBINARY(MAX) NOT NULL
);
GO

Execute the stored procedure and store the resultant model into the table

In [ ]:
-- Execute if model has not been created
DECLARE @model VARBINARY(MAX);
EXECUTE generate_model @model OUTPUT;

INSERT INTO [bdc].[hdb_py_models] (model_name, model) VALUES ('linear_model', @model)

In [ ]:
--Execute if model has already been created
DECLARE @model VARBINARY(MAX);
EXECUTE generate_py_model @model OUTPUT;

UPDATE [bdc].[hdb_py_models] SET model_name = 'linear_model', model = @model WHERE model_name = 'linear_model'

Retrieve the model from the table

In [ ]:
SELECT * FROM [bdc].[hdb_py_models] 

Create a stored procedure that retrieves the stored model and predicts a set values

In [ ]:
DROP PROCEDURE IF EXISTS py_predict_hdb;
GO
CREATE PROCEDURE py_predict_hdb (@model varchar(100))   
AS
BEGIN
    DECLARE @py_model varbinary(max) = (SELECT model FROM [bdc].[hdb_py_models] WHERE model_name = @model)

    EXECUTE sp_execute_external_script
        @language = N'Python',
        @script = N'
# Import libraries        
import pandas
import pickle
from sklearn.linear_model import LinearRegression

# Load data into dataframe
raw_dataframe = input_data
trained_model = pickle.loads(py_model)

# Data pre-processing
raw_dataframe.town = pandas.factorize(raw_dataframe.town)[0]
raw_dataframe.flat_type = pandas.factorize(raw_dataframe.flat_type)[0]
raw_dataframe.flat_model = pandas.factorize(raw_dataframe.flat_model)[0]

# Data range refinement
town_value = 0
month_value = 1
year_value = 2017
refined_df = raw_dataframe.loc[ (raw_dataframe[''town''] == town_value) &
                                (raw_dataframe[''year''] == year_value) ]

# Define dependent and independent variables
independent_variables = [''flat_type'',''floor_area_sqm'',''flat_model'',''remaining_lease_months''] #input any number of variables
dependent_variable = [''resale_price''] #only input one variable

# Split data into dependent and independent 
independent_dataset = refined_df[independent_variables]
dependent_dataset = refined_df[dependent_variable]

linear_predictions = trained_model.predict(independent_dataset)     
predictions_dataframe = pandas.DataFrame(linear_predictions)
predictions_dataframe.columns = [''predicted_price''] # appends name 

OutputDataSet = predictions_dataframe
'
, @input_data_1 = N'SELECT * FROM [bdc].[hdb_resale_prices]'
, @input_data_1_name = N'input_data'
, @params = N'@py_model varbinary(max)'
, @py_model = @py_model
WITH RESULT SETS ( ([predicted_value] INT NOT NULL) )

END;
GO

Check model score

In [ ]:
DECLARE @py_model varbinary(max) = (SELECT model FROM [bdc].[hdb_py_models] WHERE model_name = 'linear_model')

EXECUTE sp_execute_external_script
        @language = N'Python',
        @script = N'
# Import libraries        
import pandas
import pickle
from sklearn.linear_model import LinearRegression

# Load data into dataframe
raw_dataframe = input_data
trained_model = pickle.loads(py_model)

# Data pre-processing
raw_dataframe.town = pandas.factorize(raw_dataframe.town)[0]
raw_dataframe.flat_type = pandas.factorize(raw_dataframe.flat_type)[0]
raw_dataframe.flat_model = pandas.factorize(raw_dataframe.flat_model)[0]

# Data range refinement
town_value = 0
month_value = 1
year_value = 2017
refined_df = raw_dataframe.loc[ (raw_dataframe[''town''] == town_value) &
                                (raw_dataframe[''year''] == year_value) ]

# Define dependent and independent variables
independent_variables = [''flat_type'',''floor_area_sqm'',''flat_model'',''remaining_lease_months''] #input any number of variables
dependent_variable = [''resale_price''] #only input one variable

# Split data into dependent and independent 
independent_dataset = refined_df[independent_variables]
dependent_dataset = refined_df[dependent_variable]

print("Model score: ", trained_model.score(independent_dataset, dependent_dataset))
'
, @input_data_1 = N'SELECT * FROM [bdc].[hdb_resale_prices]'
, @input_data_1_name = N'input_data'
, @params = N'@py_model varbinary(max)'
, @py_model = @py_model

Create table to store predictions

In [ ]:
DROP TABLE IF EXISTS [bdc].[py_hdb_predictions]
GO

CREATE TABLE [bdc].[py_hdb_predictions](
    [ID][INT] IDENTITY(1,1) NOT NULL,
    [predicted_hdb_resale_price] [INT] NOT NULL
) ON [PRIMARY]
GO

In [ ]:
INSERT INTO [bdc].[py_hdb_predictions]
EXEC py_predict_hdb 'linear_model'

Retrieve predicted  data

In [ ]:
SELECT * FROM [bdc].[py_hdb_predictions]

In [ ]:
SELECT [bdc].[py_hdb_predictions].[predicted_hdb_resale_price], [bdc].[hdb_resale_prices].[resale_price]
FROM [bdc].[py_hdb_predictions]
JOIN  [bdc].[hdb_resale_prices]
ON [bdc].[py_hdb_predictions].[id] = [bdc].[hdb_resale_prices].[id]

In [ ]:
SELECT
[hdfs].[hdbAddress].[id],
[bdc].[hdbAddress].[town], [bdc].[hdbAddress].[year],
[mongodb].[resalePrices].[resale_price]
FROM [hdfs].[hdbAddress]
JOIN [mongodb].[resalePrices]
ON [hdfs].[hdbAddress].[id] = [mongodb].[resalePrices].[id]
WHERE [hdfs].[hdbAddress].[year] = 2017 AND [hdfs].[hdbAddress].[town] = 'ANG MO KIO'
ORDER BY [hdfs].[hdbAddress].[id] ASC

Clean up

In [ ]:
USE externalDB
IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[bdc].[hdb_py_models]') AND type in (N'U'))
DROP EXTERNAL TABLE [bdc].[hdb_py_models]
GO
IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[bdc].[hdb_predictions]') AND type in (N'U'))
DROP EXTERNAL TABLE [bdc].[hdb_predictions]
GO   
IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[bdc].[py_hdb_predictions]') AND type in (N'U'))
DROP EXTERNAL TABLE [bdc].[py_hdb_predictions]
GO  